In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from numba import jit
import math
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from fbprophet import Prophet
import pickle as pkl

train_file = '../input/hacklive-4/Train.csv'
test_file = '../input/hacklive-4/Test.csv'


df = pd.read_csv(train_file, low_memory=False, parse_dates=['Date'], index_col='ID')
tdf = pd.read_csv(test_file, low_memory=False, parse_dates=['Date'], index_col='ID')
df['train'] = 1
tdf['train'] = 0
# tdf['Open'] = tdf['Close'] = tdf['High'] = tdf['Low'] = np.float64()
stock_dfs = []
for stock in tqdm(tdf.stock.unique(), total=len(tdf.stock.unique())):
    
    forecast_dfs = []
    for val in ['Open', 'High', 'Low']:
        train_df = pd.DataFrame({'ds': df.loc[df['stock'] == stock]['Date'].values , 'y': df.loc[df['stock'] == stock][val].values})

        holidays = pd.DataFrame({'holiday': 'holiday', 'ds': np.append(df.loc[(df['stock'] == stock) & (df['holiday'] == 1 )]['Date'].values, tdf.loc[(tdf['stock'] == stock) & (tdf['holiday'] == 1 )]['Date'].values)})


        m = Prophet(yearly_seasonality=True, holidays=holidays)
        m.fit(train_df)

        future = m.make_future_dataframe(periods=41 + 3 + 17, freq='D', include_history=True)
        forecast = m.predict(future)

        
        forecast[val] = forecast.yhat
        
        
        forecast = forecast[forecast['ds'].isin(tdf.Date.values)]
        print(forecast.columns)
        forecast.drop(forecast.columns.difference([val]), 1, inplace=True)
        
        
        forecast['ID'] = tdf[tdf['stock'] == stock].index

        forecast_dfs.append(forecast)
    stock_dfs.append(forecast_dfs[0].merge(forecast_dfs[1], on='ID').merge(forecast_dfs[2], on='ID'))

    


100%|██████████| 103/103 [20:05<00:00, 11.70s/it]


In [11]:
tdf = pd.concat(stock_dfs).set_index('ID', drop=True).join(tdf)

In [12]:
tdf.shape

(4223, 8)

In [13]:
df.head()

,stock,Date,Open,High,Low,Close,holiday,unpredictability_score,train
ID,,,,,,,,,
id_0,0,2017-01-03,82.9961,82.7396,82.9144,82.8101,1,7,1
id_1,0,2017-01-04,83.1312,83.1669,83.3779,82.9690,0,7,1
id_2,0,2017-01-05,82.6622,82.7634,82.8984,82.8578,0,7,1
id_3,0,2017-01-06,83.0279,82.7950,82.8425,82.7385,0,7,1
id_4,0,2017-01-09,82.3761,82.0828,82.1473,81.8641,0,7,1


In [14]:
tdf.head()

,Open,High,Low,stock,Date,holiday,unpredictability_score,train
ID,,,,,,,,
id_713,119.744017,119.859222,119.758332,0,2019-11-01,0,7,0
id_714,120.480581,120.576431,120.508180,0,2019-11-04,0,7,0
id_715,120.840370,120.963755,120.793487,0,2019-11-05,0,7,0
id_716,120.996485,121.219422,120.968009,0,2019-11-06,0,7,0
id_717,121.391730,121.556936,121.394763,0,2019-11-07,0,7,0


In [20]:
df = pd.concat([df, tdf])

In [21]:
df.to_csv('super_train.csv')

In [22]:
from IPython.display import FileLink
FileLink('super_train.csv')

/kaggle/working/super_train.csv